In [182]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings

# Filter out warning messages
warnings.filterwarnings("ignore")

In [183]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [184]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [185]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=1000,
    height=600,
    rot=90,
    title='Cryptocurrency Market Data: Returns for Several Holding Periods (%)',
    ylabel= 'Return (%)',
    xlabel='Cryptocurrency Name',
    value_label='Return in the Period (%)'
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (Return in the Period (%))

The raw data reflects different cryptocurrencies' price change percentages in 24 hours, 7 days, 14 days, 30 days, 60 days, 200 days, and 1 year time periods. These returns will be standarized (below) before starting the analysis.

### Prepare the Data

In [186]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
crypto_scaled_data = StandardScaler().fit_transform(df_market_data)
crypto_scaled_data

array([[ 5.08529366e-01,  4.93193071e-01,  7.72200433e-01,
         2.35459633e-01, -6.74950963e-02, -3.55953481e-01,
        -2.51636882e-01],
       [ 1.85445894e-01,  9.34445040e-01,  5.58692121e-01,
        -5.43409317e-02, -2.73482725e-01, -1.15759474e-01,
        -1.99352110e-01],
       [ 2.17739616e-02, -7.06336853e-01, -2.16804207e-02,
        -6.10301536e-02,  8.00452481e-03, -5.50246924e-01,
        -2.82060506e-01],
       [-4.07643829e-02, -8.10928066e-01,  2.49457974e-01,
        -5.03879651e-02, -3.73164019e-01, -4.58258816e-01,
        -2.95546142e-01],
       [ 1.19303608e+00,  2.00095907e+00,  1.76061001e+00,
         5.45842065e-01, -2.91202870e-01, -4.99847761e-01,
        -2.70316950e-01],
       [ 8.91870708e-01,  1.32729453e+00,  8.00214184e-01,
        -5.71478992e-02,  7.78653106e-01, -1.88231917e-01,
        -2.25532605e-01],
       [ 1.13972400e-02,  2.57225091e+00,  1.10164693e+00,
        -4.90495415e-01, -9.31954023e-01,  3.87758986e-01,
        -1.8284399

In [187]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(crypto_scaled_data,columns=df_market_data.columns)

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


In [188]:
# Plot the scaled data
df_market_data_scaled.hvplot.line(
    width=1000,
    height=600,
    rot=90,
    title='Cryptocurrency Market Data Scaled: Standarized Returns for Several Holding Periods (%)',
    ylabel= 'Standarized Return (%)',
    xlabel='Cryptocurrency Name',
    value_label='Standarized Return in the Period (%)'
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (Standarized Return in the Period (%))

---

### Find the Best Value for k Using the Original Data.

In [189]:
# Create a list with the number of k-values from 1 to 11
k = range(1,12)
list(k)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [190]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    k_mean = KMeans(n_clusters=i, random_state=2, n_init='auto')
    k_mean.fit(df_market_data_scaled)
    inertia.append(k_mean.inertia_)
    
inertia

[287.0,
 212.12334207486256,
 166.9783904246291,
 91.1488896797012,
 79.46389757576232,
 55.418857865018374,
 47.33347983716501,
 39.61330681914663,
 35.28561402540515,
 31.11998729411641,
 27.130977375244992]

In [191]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data= {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()

,k,inertia
0,1,287.000000
1,2,212.123342
2,3,166.978390
3,4,91.148890
4,5,79.463898


In [192]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot=df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    xticks=k,
    title="Standarized Performance Data: Inertia versus Number of Clusters", 
    xlabel = 'Number of Clusters (k)',
    ylabel = 'Inertia (units)'
)
elbow_plot

:Curve   [k]   (inertia)

In [193]:
# Elbow Rule is demonstrated graphically above. To best determine number of clusters numerically calculate the following:
# Rate of decrease of the inertia
# Rate of intertia for k, with the respect to only one cluster (k=1)
df_elbow['inertia_fraction_with_respect_to_one_cluster'] = df_elbow['inertia']/df_elbow.iloc[0,1]

# Rate of decrease from one level to the next in the initial inertia (k=1). This is: inertia_rate(k)-inertia_rate(k-1)
df_elbow['inertia_rate_of_decrease'] = df_elbow['inertia_fraction_with_respect_to_one_cluster'] - df_elbow['inertia_fraction_with_respect_to_one_cluster'].shift()
df_elbow

,k,inertia,inertia_fraction_with_respect_to_one_cluster,inertia_rate_of_decrease
0,1,287.000000,1.000000,NaN
1,2,212.123342,0.739106,-0.260894
2,3,166.978390,0.581806,-0.157299
3,4,91.148890,0.317592,-0.264214
4,5,79.463898,0.276878,-0.040714
5,6,55.418858,0.193097,-0.083781
6,7,47.333480,0.164925,-0.028172
7,8,39.613307,0.138025,-0.026900
8,9,35.285614,0.122946,-0.015079
9,10,31.119987,0.108432,-0.014514


#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** Best value of K (number of clusters) is 4, where the largest decrease (26.4%) in "inertia" occurs.

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled Data

In [194]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1, n_init="auto")

In [195]:
# Fit the K-Means model using the scaled data
model.fit(df_market_data_scaled)

KMeans(n_clusters=4, n_init='auto', random_state=1)

In [196]:
# Predict the clusters to group the cryptocurrencies using the scaled data
k4_clusters = model.predict(df_market_data_scaled)

# Print the resulting array of cluster values.
print(k4_clusters)

[1 1 2 2 1 1 1 1 2 2 0 2 2 1 2 2 2 2 1 2 0 1 2 2 2 2 2 0 1 2 2 2 3 2 0 0 1
 0 2 2 0]


In [197]:
# Create a copy of the DataFrame
market_data_predictions = df_market_data_scaled.copy()

In [198]:
# Add a new column to the DataFrame with the predicted clusters
market_data_predictions["crypto_clusters_original"] = k4_clusters

# Display sample data
market_data_predictions.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,crypto_clusters_original
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,1
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,1
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,2
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,2
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,1


In [199]:
# Generate summary statistics
market_data_predictions.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,crypto_clusters_original
count,41.000000,4.100000e+01,4.100000e+01,4.100000e+01,4.100000e+01,4.100000e+01,4.100000e+01,41.000000
mean,0.000000,1.895503e-16,2.707861e-17,2.978647e-17,-5.415722e-18,-1.326852e-16,4.197185e-17,1.414634
std,1.012423,1.012423e+00,1.012423e+00,1.012423e+00,1.012423e+00,1.012423e+00,1.012423e+00,0.805469
min,-4.981042,-1.682027e+00,-2.217108e+00,-1.393153e+00,-9.560487e-01,-5.511464e-01,-2.963296e-01,0.000000
25%,-0.127467,-7.066688e-01,-6.299628e-01,-4.605580e-01,-5.517599e-01,-4.998478e-01,-2.817468e-01,1.000000
50%,0.077497,-1.906843e-01,-9.190922e-03,-6.103015e-02,-1.592496e-01,-3.550537e-01,-2.255326e-01,2.000000
75%,0.331280,4.931931e-01,6.435649e-01,1.165382e-01,1.606038e-02,-4.736110e-02,-1.454693e-01,2.000000
max,1.919812,2.572251e+00,2.907054e+00,5.351455e+00,4.769913e+00,4.632380e+00,6.088625e+00,3.000000


In [200]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

market_predictions_scatterplot = market_data_predictions.hvplot.scatter(
    x     = "price_change_percentage_24h", 
    y     = "price_change_percentage_7d", 
    by    = "crypto_clusters_original",
    title = "Cryptocurrencies Standardized Price Change Percentages. K-Mean Clusters with k=4.",
    hover_cols = "coin_id",
    frame_width=700,
    frame_height=500 
)

market_predictions_scatterplot

:NdOverlay   [crypto_clusters_original]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [201]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


In [202]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
crypto_pca_data = pca.fit_transform(df_market_data_scaled)

# View the first five rows of the DataFrame. 
crypto_pca_data[0:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [203]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
explained_variance = pca.explained_variance_ratio_

for i, variance in enumerate(explained_variance, start=1):
    print(f"Explained Variance for PC{i}: {variance:.4f}")

Explained Variance for PC1: 0.3720
Explained Variance for PC2: 0.3470
Explained Variance for PC3: 0.1760


In [204]:
# Determine how much information can be attributed to the three principal components.
total_explained_variance = sum(explained_variance)
print(f"The Total Explained Variance of the three principal components is: {total_explained_variance * 100:.4f}%")

The Total Explained Variance of the three principal components is: 89.5032%


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the three principal components is 89.5%.

In [205]:
# Create a new DataFrame with the PCA data.
# Creating a DataFrame with the PCA data
df_market_data_pca = pd.DataFrame(crypto_pca_data, columns=['PC1', 'PC2', 'PC3'])

# Copy the crypto names from the original data
df_market_data_pca['coin_id'] = df_market_data.index

# Set the coinid column as index
df_market_data_pca = df_market_data_pca.set_index('coin_id')

# Display sample data
df_market_data_pca.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [206]:
# Create a list with the number of k-values from 1 to 11
pca_k = range(1,12)
list(pca_k)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [207]:
# Create an empty list to store the inertia values
pca_inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in pca_k:
    k_mean = KMeans(n_clusters=i, random_state=2, n_init='auto')
    k_mean.fit(df_market_data_pca)
    pca_inertia.append(k_mean.inertia_)
    
pca_inertia

[256.87408556789256,
 182.33953007775648,
 140.43183298408374,
 49.665496651797355,
 39.174438685096064,
 32.18253017832155,
 26.872035396380944,
 19.45174720604135,
 13.814081971558094,
 11.13196312954012,
 8.130111899790265]

In [208]:
# Create a dictionary with the data to plot the Elbow curve
pca_elbow_data = {'k': pca_k,'Inertia': pca_inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_pca_elbow = pd.DataFrame(pca_elbow_data)

# Review the DataFrame
df_pca_elbow.head()

,k,Inertia
0,1,256.874086
1,2,182.339530
2,3,140.431833
3,4,49.665497
4,5,39.174439


In [209]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
pca_elbow_plot = df_pca_elbow.hvplot(x='k', y='Inertia', xticks=k, title='Principal Component Data: Inertia versus Number of Clusters', 
                                     xlabel = 'Number of Clusters (k)', ylabel = 'Inertia (units)', frame_width=800, frame_height=400)
pca_elbow_plot

:Curve   [k]   (Inertia)

In [210]:
# Elbow Rule is demonstrated graphically above. To best determine number of clusters numerically calculate the following:
# Rate of decrease of the inertia
# Rate of intertia for k, with the respect to only one cluster (k=1)
df_pca_elbow['inertia_fraction_with_respect_to_one_cluster'] = df_pca_elbow['Inertia']/df_pca_elbow.iloc[0,1]

# Rate of decrease from one level to the next in the initial inertia (k=1). This is: inertia_rate(k)-inertia_rate(k-1)
df_pca_elbow['inertia_rate_of_decrease'] = df_pca_elbow['inertia_fraction_with_respect_to_one_cluster'] - df_pca_elbow['inertia_fraction_with_respect_to_one_cluster'].shift()
df_pca_elbow

,k,Inertia,inertia_fraction_with_respect_to_one_cluster,inertia_rate_of_decrease
0,1,256.874086,1.000000,NaN
1,2,182.339530,0.709840,-0.290160
2,3,140.431833,0.546695,-0.163145
3,4,49.665497,0.193346,-0.353350
4,5,39.174439,0.152504,-0.040841
5,6,32.182530,0.125285,-0.027219
6,7,26.872035,0.104612,-0.020674
7,8,19.451747,0.075725,-0.028887
8,9,13.814082,0.053778,-0.021947
9,10,11.131963,0.043336,-0.010441


#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** Best value of K (number of clusters) is 4, where the largest decrease in "inertia" occurs.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, the best number of clusters K is 4 for both the original standardized data and PCA data, even though the shape of the elbow curves are slightly different.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [211]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1, n_init="auto")

In [212]:
# Fit the K-Means model using the PCA data
model.fit(df_market_data_pca)

KMeans(n_clusters=4, n_init='auto', random_state=1)

In [213]:
# Predict the clusters to group the cryptocurrencies using the PCA data
crypto_pca_clusters = model.predict(df_market_data_pca)

# Print the resulting array of cluster values.
print(crypto_pca_clusters)

[1 1 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 2 1 0 0 3
 0 0 0 0]


In [214]:
# Create a copy of the DataFrame with the PCA data
market_data_pca_predictions = df_market_data_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
market_data_pca_predictions['CryptoClusters'] = crypto_pca_clusters

# Display sample data
market_data_pca_predictions.head()

,PC1,PC2,PC3,CryptoClusters
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,1
ethereum,-0.458261,0.458466,0.952877,1
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,1


In [215]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_predictions_scatter_plot=market_data_pca_predictions.hvplot.scatter(
    x='PC1',
    y='PC2',
    by='CryptoClusters',
    hover_cols=['coin_id'],
    title='Cryptocurrencies using the PCA data. K-Mean Clusters with k=4.',
    cmap="viridis",
    frame_width=700,
    frame_height=500
)
pca_predictions_scatter_plot

:NdOverlay   [CryptoClusters]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [216]:
# Composite plot to contrast the Elbow curves
elbow_plot + pca_elbow_plot

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (Inertia)

In [217]:
# Composite plot to contrast the clusters
market_predictions_scatterplot + pca_predictions_scatter_plot

:Layout
   .NdOverlay.I  :NdOverlay   [crypto_clusters_original]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [CryptoClusters]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Even though we reduced the number of features from 7 for the standardized original data to 3 features using the Principal Components Analysis (PCA) approach, we still determined the same number of clusters, which was 4, as the optimal number of clusters.  The PCA generated a higher inertia decrease at 4 clusters than the 4 clusters' solution for the standardized data (35.3% vs. 26.4%, respectively).  Furthermore, the PCA clustering approach resulted in more separation among the 4 clusters, and specifically the two outlier clusters (2 and 3 in PCA graph).